<p style="font-family: Arial; font-size:2.75em;color:blue; font-style:bold"><br>
Projet 6 :<br><br>
Catégorisez automatiquement des questions<br><br>
</p><br>

* **Data Source / Data Source :** 
    - Data :
        - Outils d'extraction de data : https://data.stackexchange.com/stackoverflow
        - Script : https://github.com/EricJacquesPro/TextCategorization/blob/master/SQL/ExtractionDonnees.sql
    - Source Python : https://github.com/EricJacquesPro/TextCategorization
* **Description / Description : ** Projet 6 de la formation Data Scientist Proposée par OpenClassroom
    - Analyse Supervised
    - Entrainement Supervised
* **Auteur / Author : ** Eric JACQUES 
* **Date : ** 16/06/2020 

<p style="font-family: Arial; font-size:1.75em;color:#2462C0; font-style:bold"><br>
I - Sélection des datas entrantes</p>

In [1]:
def scoring(x_test, y_true, clf, lb, mode_supervise_with_lda = False, lda_model = None):
    debut_scoring = time.time()
    nb_tag_1 = 0.0
    nb_tag_5 = 0.0
    classes = lb.classes_
    print(x_test.shape[0])
    no_top_words = 5
    for i in range(x_test.shape[0]):

        debut_boucle = time.time()
        #for i in range(x_test.shape[0]):
        text_projection = x_test
        if(mode_supervise_with_lda):
            text_projection = lda_model.transform(x_test[i])
        '''
        print (text_projection)
        print (text_projection.shape)
        '''
        #voir le temps
        #voir nb tag
        
        debut_prediction = time.time()
        predicted = clf.predict_proba(text_projection)
        fin_prediction = time.time()
        print("temps pour la prediction : {0} s".format(fin_prediction - debut_prediction))
        del debut_prediction, fin_prediction

        '''
        print (predicted)
        print (len(predicted))
        '''
        debut_generation_tag = time.time()
        tempTag = [(1-item[0][0]) for item in predicted]
        list_id = [[i, x] for i, x in enumerate(tempTag) if x > 0.0050]
        '''
        tempTag = predicted[0]
        list_id = [[i, x] for i, x in enumerate(tempTag) ]#if x > 0.0050]
        '''
        '''
        print (predicted)
        print (predicted.shape)
        '''
        list_id_sorted = sorted(list_id, reverse=True, key=lambda x: x[1])
        #print (list_id_sorted)

        list_id_sorted_suggested = [x[0] for i, x in enumerate(list_id_sorted[:-no_top_words - 1:-1])]
        #print(list_id_sorted_suggested)
        prediction = [classes[id] for id in list_id_sorted_suggested]
        fin_generation_tag = time.time()
        print("temps pour la generation : {0} s".format(fin_generation_tag - debut_generation_tag))
        del debut_generation_tag, fin_generation_tag

        #print (str(prediction))
        
        debut_y = time.time()
        l_y = [[i, x] for i, x in enumerate(y_true[i]) if x > 0]
        l_y_tagged = [x[0] for i, x in enumerate(l_y[:-no_top_words - 1:-1])]
        l_y_tags = [classes[id] for id in l_y_tagged]
        #print (l_y_tags)
        fin_y = time.time()
        print("temps pour lecture y : {0} s".format(fin_y - debut_y))
        del debut_y, fin_y

        debut_ch1 = time.time()
        check_1 = False
        check_1 = any(item in prediction for item in l_y_tags)

        if check_1 is True:
            nb_tag_1 = nb_tag_1 + 1
        """    print("The list {} contains some elements of the list {}".format(prediction, l_y_tags))    
        else :
            print("No, List1 doesn't have any elements of the List2.")
        """
        fin_ch1 = time.time()
        print("temps pour check 1 : {0} s".format(fin_ch1 - debut_ch1))
        del debut_ch1, fin_ch1
        
        debut_ch5 = time.time()
        check_5 = False
        check_5 = all(item in prediction for item in l_y_tags)
        if check_5 is True:
            nb_tag_5 = nb_tag_5 + 1
        """    print("The list {} contains all elements of the list {}".format(prediction, l_y_tags))    
        else :
            print("No, List1 doesn't have any elements of the List2.")
        """
        fin_ch5 = time.time()
        print("temps pour check 5 : {0} s".format(fin_ch5 - debut_ch5))
        del debut_ch5, fin_ch5
        #str([tag for tag in y_true[i]]if tag ==1)
        fin_boucle = time.time()
        print("temps pour la boucle scorring : {0} s".format(fin_boucle - debut_boucle))
        del fin_boucle, debut_boucle
    fin_scoring = time.time()
    print("temps pour le function scoring : {0} s".format(fin_scoring - debut_scoring))
    del fin_scoring, debut_scoring
    return nb_tag_1, (100.0 * nb_tag_1 / float(x_test.shape[0])), nb_tag_5, (100.0 * nb_tag_5 / float(x_test.shape[0]))


In [2]:

import time

In [3]:
import sys
import os
import numpy as np
import pandas as pd
sys.path.insert(0, './Python/')
'''
python class for the data engineering (reading, cleaning, training...)
localised in ./Python
'''
from tagText import TagText 
tagText = TagText()

tagText.nombre_post_entree = 50000#500#00
tagText.precision = 50000#500#00

print(tagText.urlDirectory)#folder for cvs file generated by SQL query (cf. I) 
print(tagText.fileName)#name of the cvs file generated by SQL query (cf. I)
debut_lecture = time.time()
data_question = tagText.read_source()
print(data_question.head())

fin_lecture = time.time()
print("temps pour la lecture : {0} s".format(fin_lecture - debut_lecture))
del fin_lecture, debut_lecture

debut_preprocess = time.time()
data_preprocessed = data_question['body'].apply(tagText.preprocessing)
fin_preprocess = time.time()
print("temps pour le pre processing : {0} s".format(fin_preprocess - debut_preprocess))
del fin_preprocess, debut_preprocess

print(data_preprocessed.head())

C:\Users\naru_\Anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Data/
QuestionVsTags.csv
                                                body  \
0  <p>Here is a piece of C++ code that shows some...   
1  <p>I accidentally committed the wrong files to...   
2  <p>I want to delete a branch both locally and ...   
3  <p>What are the differences between <code>git ...   
4  <p>What is the use of the <code>yield</code> k...   

                                               title  \
0  Why is processing a sorted array faster than p...   
1  How do I undo the most recent local commits in...   
2  How do I delete a Git branch locally and remot...   
3  What is the difference between 'git pull' and ...   
4                  What does the "yield" keyword do?   

                                                tags  \
0  <java><c++><performance><optimization><branch-...   
1  <git><version-control><git-commit><undo><pre-c...   
2  <git><version-control><git-branch><git-push><g...   
3        <git><version-control><git-pull><git-fetch>   
4    <python><iterato

In [4]:
def supervised_prepare_tagV2(self, data_preprocessed, data_tag):
    '''
    prepare classifier and class from file for supervised model
    '''
    from sklearn.pipeline import Pipeline
    from sklearn import metrics
    from sklearn.model_selection import train_test_split
    from sklearn.feature_extraction.text import TfidfVectorizer

    y_all = [
        item[:-1].split(',')#-1 car il y a un ',' à la fin de la ligne
        for item in data_tag
    ]

    #print(y_train_tag)
    lb = tagText.MultiLabelBinarizer()
    Y_all = lb.fit_transform(y_all)


    # 80/20 split
    X_train, X_test, y_train, y_test = train_test_split(
        data_preprocessed, y_all, test_size=0.05,train_size=0.95, random_state=0)
    y_train = lb.transform(y_train)
    y_test = lb.transform(y_test)

    classifier = Pipeline([
        ('vectorizer', self.CountVectorizer()),
        ('tfidf', self.TfidfTransformer()),
        (
            'clf',
            self.RandomForestClassifier(
                n_estimators=100,
                max_depth=2,
                random_state=0
            )
        )
    ])  
    classifier.fit(X_train, y_train)
    '''
    print (len(X_test))
    print (len(y_test))
    print (y_test[0])
    print (X_test)
    '''
    nb1, score1, nb5, score5 = scoring(X_test, y_test, classifier, lb, False, None)
    
    print("score 1 occurence {}".format(score1))
    print("score 5 occurences {}".format(score5))
    
    return classifier, lb.classes_


In [ ]:
classifier, tags = supervised_prepare_tagV2(tagText, data_preprocessed, data_question['SelectedTags'])
#del classifier, tags

2500
temps pour la prediction : 19.09520721435547 s
temps pour la generation : 0.0 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 19.09520721435547 s
temps pour la prediction : 19.17697834968567 s
temps pour la generation : 0.0 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 19.17697834968567 s
temps pour la prediction : 19.105719566345215 s
temps pour la generation : 0.0 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 19.105719566345215 s
temps pour la prediction : 19.09343385696411 s
temps pour la generation : 0.0 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 19.09343385696411 s
temps pour la prediction : 19.09005904197693 s
temps pour la generation : 0.0 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0

temps pour la prediction : 19.2940034866333 s
temps pour la generation : 0.0 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 19.2940034866333 s
temps pour la prediction : 19.56214928627014 s
temps pour la generation : 0.0 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 19.56214928627014 s
temps pour la prediction : 21.4024875164032 s
temps pour la generation : 0.0 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 21.4024875164032 s
temps pour la prediction : 19.4227614402771 s
temps pour la generation : 0.0 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 19.4227614402771 s
temps pour la prediction : 19.17087173461914 s
temps pour la generation : 0.0 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour

temps pour la prediction : 19.32795476913452 s
temps pour la generation : 0.0 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 19.32795476913452 s
temps pour la prediction : 19.187505960464478 s
temps pour la generation : 0.0 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 19.187505960464478 s
temps pour la prediction : 19.26663899421692 s
temps pour la generation : 0.0 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 19.26663899421692 s
temps pour la prediction : 19.20832633972168 s
temps pour la generation : 0.015571832656860352 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 19.22389817237854 s
temps pour la prediction : 19.290401220321655 s
temps pour la generation : 0.0 s
temps pour lecture y : 0.0 s
temps pour 

temps pour la prediction : 19.165541410446167 s
temps pour la generation : 0.0 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 19.165541410446167 s
temps pour la prediction : 19.23605227470398 s
temps pour la generation : 0.0 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 19.23605227470398 s
temps pour la prediction : 19.192220449447632 s
temps pour la generation : 0.015582799911499023 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 19.20780324935913 s
temps pour la prediction : 19.196342706680298 s
temps pour la generation : 0.0 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 19.196342706680298 s
temps pour la prediction : 19.241185665130615 s
temps pour la generation : 0.0 s
temps pour lecture y : 0.0 s
temps po

temps pour la prediction : 19.222700119018555 s
temps pour la generation : 0.0 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 19.222700119018555 s
temps pour la prediction : 19.120963096618652 s
temps pour la generation : 0.0 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 19.120963096618652 s
temps pour la prediction : 19.201573371887207 s
temps pour la generation : 0.0 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 19.201573371887207 s
temps pour la prediction : 19.183958768844604 s
temps pour la generation : 0.0 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 19.183958768844604 s
temps pour la prediction : 19.22815990447998 s
temps pour la generation : 0.0 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.

...

temps pour la prediction : 20.11093282699585 s
temps pour la generation : 0.0 s
temps pour lecture y : 0.000997781753540039 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 20.11193060874939 s
temps pour la prediction : 19.9259512424469 s
temps pour la generation : 0.0009984970092773438 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 19.926949739456177 s
temps pour la prediction : 20.05822205543518 s
temps pour la generation : 0.0 s
temps pour lecture y : 0.0009984970092773438 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 20.059220552444458 s
temps pour la prediction : 20.168150424957275 s
temps pour la generation : 0.0009980201721191406 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 20.169148445129395 s
temps pour la prediction : 20.041045904159546 s
temps pour la gene

temps pour la prediction : 19.932337522506714 s
temps pour la generation : 0.0009980201721191406 s
temps pour lecture y : 0.0009975433349609375 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 19.934333086013794 s
temps pour la prediction : 20.317970514297485 s
temps pour la generation : 0.0009982585906982422 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 20.318968772888184 s
temps pour la prediction : 20.299197912216187 s
temps pour la generation : 0.0 s
temps pour lecture y : 0.000997304916381836 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 20.30019521713257 s
temps pour la prediction : 20.35840368270874 s
temps pour la generation : 0.0009980201721191406 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 20.35940170288086 s
temps pour la prediction : 20.38980484008789 s

temps pour la prediction : 20.229166269302368 s
temps pour la generation : 0.000997304916381836 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 20.23016357421875 s
temps pour la prediction : 20.133714199066162 s
temps pour la generation : 0.000997304916381836 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 20.134711503982544 s
temps pour la prediction : 20.186819791793823 s
temps pour la generation : 0.0019953250885009766 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 20.188815116882324 s
temps pour la prediction : 20.1316237449646 s
temps pour la generation : 0.000997304916381836 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 20.13262104988098 s
temps pour la prediction : 20.21993374824524 s
temps pour la genera

temps pour la prediction : 20.023972511291504 s
temps pour la generation : 0.0009980201721191406 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 20.024970531463623 s
temps pour la prediction : 20.067009210586548 s
temps pour la generation : 0.0009975433349609375 s
temps pour lecture y : 0.000997781753540039 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 20.06900453567505 s
temps pour la prediction : 19.94948172569275 s
temps pour la generation : 0.0009980201721191406 s
temps pour lecture y : 0.0009975433349609375 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 19.95147728919983 s
temps pour la prediction : 19.93483304977417 s
temps pour la generation : 0.0009975433349609375 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 19.93583059310913 s
temps pour la prediction : 20.

temps pour la prediction : 19.980031967163086 s
temps pour la generation : 0.0 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 19.981029510498047 s
temps pour la prediction : 20.02224063873291 s
temps pour la generation : 0.0009975433349609375 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 20.02323818206787 s
temps pour la prediction : 20.18103051185608 s
temps pour la generation : 0.0 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 20.18103051185608 s
temps pour la prediction : 20.17271137237549 s
temps pour la generation : 0.0 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 20.17370867729187 s
temps pour la prediction : 20.3683500289917 s
temps pour la generation : 0.0 s
temps pour lecture y : 0.0 s
temps pour c

In [ ]:
classifier, tags = tagText.supervised_prepare_tagV2(data_preprocessed, data_question['SelectedTags'])
del classifier, tags



10000
10000
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 

In [ ]:

message = 'git est un outils de code source. C\'est bien'
print(message)



print(tagText.supervised_predict(message, classifier, tags))

In [ ]:
print(tagText.supervised_predict(data_preprocessed[0], classifier, tags ))

In [ ]:
print (tags)

In [ ]:
tagText.supervised_predict('c c# .Net', classifier, tags )

In [ ]:
tagText.supervised_predict('how to convert int in flaot in c c# .Net', classifier, tags, 120000 )